# Maven Analytics Challenge May 2024
## UK National Rail

https://mavenanalytics.io/challenges/maven-rail-challenge/08941141-d23f-4cc9-93a3-4c25ed06e1c3

For the Maven Rail Challenge, you'll play the role of a BI Developer for National Rail, a company that provides business services to passenger train operators in England, Scotland, and Wales.

You've been asked by your manager to create an exploratory dashboard that helps them:

- Identify the most popular routes
- Determine peak travel times
- Analyze revenue from different ticket types & classes
- Diagnose on-time performance and contributing factors

In [7]:
import pandas as pd
import numpy as np

Import the two raw data files into dataframes

In [8]:
# read the railway.csv data from the data folder into a dataframe called df_railway
df_railway = pd.read_csv('../data/railway.csv')
df_railway.head()

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,Liverpool Lime Street,2024-01-01,11:00:00,13:30:00,13:30:00,On Time,NaN,No
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,York,2024-01-01,09:45:00,11:35:00,11:40:00,Delayed,Signal Failure,No
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,NaN,Standard,Advance,3,Liverpool Lime Street,Manchester Piccadilly,2024-01-02,18:15:00,18:45:00,18:45:00,On Time,NaN,No
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,NaN,Standard,Advance,13,London Paddington,Reading,2024-01-01,21:30:00,22:30:00,22:30:00,On Time,NaN,No
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,NaN,Standard,Advance,76,Liverpool Lime Street,London Euston,2024-01-01,16:45:00,19:00:00,19:00:00,On Time,NaN,No


Inspect the data dictionary

In [9]:
# import railway_data_dictionary.csv from the data folder into a dataframe called df_data_dict
df_data_dict = pd.read_csv('../data/railway_data_dictionary.csv')

# display the full width of the dataframe
pd.set_option('display.max_colwidth', None)

# display the dataframe
df_data_dict

,Field,Description
0,Transaction ID,Unique identifier for an individual train ticket purchase
1,Date of Purchase,Date the ticket was purchased
2,Time of Purchase,Time the ticket was purchased
3,Purchase Type,Whether the ticket was purchased online or directly at a train station
4,Payment Method,"Payment method used to purchase the ticket (Contactles, Credit Card, or Debit Card)"
5,Railcard,"Whether the passenger is a National Railcard holder (Adult, Senior, or Disabled) or not (None). Railcard holders get 1/3 off their ticket purchases."
6,Ticket Class,Seat class for the ticket (Standard or First)
7,Ticket Type,When you bought or can use the ticket. Advance tickets are 1/2 off and must be purchased at least a day prior to departure. Off-Peak tickets are 1/4 off and must be used outside of peak hours (weekdays between 6-8am and 4-6pm). Anytime tickets are full price and can be bought and used at any time during the day.
8,Price,Final cost of the ticket
9,Departure Station,Station to board the train


Inspect the incidence of NaNs

In [10]:
# display the counts of NaN in the dataframe
nan_counts = df_railway.isnull().sum()
nan_counts[nan_counts > 0]

Railcard               20918
Actual Arrival Time     1880
Reason for Delay       27481
dtype: int64

In [11]:
df_railway['Railcard'].fillna('None').value_counts()

Railcard
None        20918
Adult        4846
Disabled     3089
Senior       2800
Name: count, dtype: int64

In [12]:
# fill the Railcard column with 'None' where there are NaN values
df_railway['Railcard'] = df_railway['Railcard'].fillna('None')
df_railway['Railcard'].value_counts()

Railcard
None        20918
Adult        4846
Disabled     3089
Senior       2800
Name: count, dtype: int64

In [13]:
df_railway['Reason for Delay'].fillna('n/a').value_counts()

Reason for Delay
n/a                   27481
Weather                 995
Technical Issue         707
Signal Failure          523
Signal failure          447
Staffing                410
Staff Shortage          399
Weather Conditions      377
Traffic                 314
Name: count, dtype: int64

In [14]:
# fill the Railcard column with 'None' where there are NaN values
df_railway['Reason for Delay'] = df_railway['Reason for Delay'].fillna('Not delayed')
df_railway['Reason for Delay'].value_counts()

Reason for Delay
Not delayed           27481
Weather                 995
Technical Issue         707
Signal Failure          523
Signal failure          447
Staffing                410
Staff Shortage          399
Weather Conditions      377
Traffic                 314
Name: count, dtype: int64

In [15]:
# display the counts of NaN in the dataframe
nan_counts = df_railway.isnull().sum()
nan_counts[nan_counts > 0]

Actual Arrival Time    1880
dtype: int64

In [16]:
# show a random sample of rows where the Actual Arrival Time is NaN
df_railway[df_railway['Actual Arrival Time'].isnull()].sample(5)

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request
18443,f6948b5f-d3d9-4df2-924b,2024-03-11,07:48:17,Online,Debit Card,Senior,Standard,Advance,8,London Paddington,Reading,2024-03-12,06:15:00,07:15:00,NaN,Cancelled,Technical Issue,Yes
19197,fe523d44-bcff-4e78-b5e9,2024-03-14,03:28:33,Online,Contactless,None,Standard,Advance,8,York,Durham,2024-03-15,01:45:00,02:35:00,NaN,Cancelled,Weather,Yes
21965,096d0f88-5fb0-4265-b5bc,2024-03-24,11:55:31,Online,Credit Card,None,Standard,Off-Peak,10,London Euston,Birmingham New Street,2024-03-24,13:15:00,14:35:00,NaN,Cancelled,Signal failure,No
7951,ba49cd0d-c9af-4123-b4f3,2024-01-30,07:29:31,Online,Credit Card,None,Standard,Anytime,35,London Paddington,Oxford,2024-01-30,07:45:00,09:15:00,NaN,Cancelled,Traffic,No
30829,4cf0a286-c0a5-4ff7-82f9,2024-04-27,11:10:26,Station,Contactless,None,Standard,Off-Peak,10,London Euston,Birmingham New Street,2024-04-27,12:30:00,13:50:00,NaN,Cancelled,Signal Failure,No


In [17]:
# show the value counts of Journey Status for rows where Actual Arrival Time is NaN
df_railway[df_railway['Actual Arrival Time'].isnull()]['Journey Status'].value_counts()

Journey Status
Cancelled    1880
Name: count, dtype: int64

In [18]:
# show any rows where the Date of Journey is before the Date of Purchase
df_railway[df_railway['Date of Journey'] < df_railway['Date of Purchase']]

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request


In [19]:
# show any rows where the Departure Station is the same as the Arrival Destination
df_railway[df_railway['Departure Station'] == df_railway['Arrival Destination']]

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request


In [20]:
# create a 'Route' column that combines the Departure Station and Arrival Destination, separated with a dash
df_railway['Route'] = df_railway['Departure Station'] + ' - ' + df_railway['Arrival Destination']
df_railway['Route'].head()

0        London Paddington - Liverpool Lime Street
1                        London Kings Cross - York
2    Liverpool Lime Street - Manchester Piccadilly
3                      London Paddington - Reading
4            Liverpool Lime Street - London Euston
Name: Route, dtype: object

In [21]:
# set the notebook to display all rows of the dataframe
pd.set_option('display.max_rows', None)

# produce descriptive statistics of Price for each Route. Show all rows. 
route_prices_stats = df_railway.groupby('Route')['Price'].describe()
route_prices_stats

,count,mean,std,min,25%,50%,75%,max
Route,,,,,,,,
Birmingham New Street - Coventry,65.0,4.138462,1.210332,3.0,3.0,4.0,5.00,8.0
Birmingham New Street - Edinburgh,16.0,49.875000,25.120709,36.0,36.0,36.0,54.00,111.0
Birmingham New Street - Liverpool Lime Street,175.0,10.228571,13.787563,4.0,4.0,6.0,8.00,77.0
Birmingham New Street - London Euston,125.0,28.752000,15.520879,14.0,21.0,21.0,31.00,109.0
Birmingham New Street - London Kings Cross,17.0,31.470588,23.076280,15.0,15.0,29.0,29.00,78.0
Birmingham New Street - London Paddington,32.0,22.437500,16.453135,13.0,13.0,13.0,25.00,78.0
Birmingham New Street - London St Pancras,702.0,27.081197,15.793079,15.0,15.0,22.0,29.00,117.0
Birmingham New Street - Manchester Piccadilly,224.0,11.191964,5.149228,7.0,7.0,11.0,15.00,38.0
Birmingham New Street - Nuneaton,219.0,9.972603,4.271645,7.0,7.0,7.0,13.00,28.0


In [22]:
# show the value counts of refund request
df_railway['Refund Request'].value_counts()

Refund Request
No     30535
Yes     1118
Name: count, dtype: int64

In [24]:
# count the number of rows per transaction id, then show the first 5 rows of transactions that had more than one row
transaction_counts = df_railway['Transaction ID'].value_counts()
transaction_counts[transaction_counts > 1]

Series([], Name: count, dtype: int64)

In [23]:
# write df_railway to a new csv file called railway_cleaned.csv in the data folder
df_railway.to_csv('../data/railway_cleaned.csv', index=False)